# Homework 3

## Description

### Data
[Reuters-21578 Text Categorization Collection Data Set](https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection) - about 2 million instances, 28MB in size


News: 21 SGML files
We only deal with news contents inside <body> </body> tags
The other files will not be needed in this homework

### Format
One text file consisting of lines of records.

Each record contains 9 attributes separated by semicolons: 


Reuters-21578, Distribution 1.0 includes five files (all-exchanges-strings.lc.txt, all-orgs-strings.lc.txt, all-people-strings.lc.txt, all-places-strings.lc.txt, and all-topics-strings.lc.txt) which list the names of *all* legal categories in each set. A sixth file, cat-descriptions_120396.txt gives some additional information on the category sets.


### Task
4 subtasks:
+ (30pt) Given the Reuters-21578 dataset, please calculate all k-shingles and output the set representation of the text dataset as a matrix.
+ (30pt) Given the set representation, compute the minhash signatures of all documents using MapReduce.
+ (40pt) Implement the LSH algorithm by MapReduce and output the resulting candidate pairs of similar documents. 
+ (Bonus) Implement K-nearest neighbor (KNN) search using LSH and compare its performance with linear search.

## Output Format

1. Set representation:
A MxN matrix: with rows as shingles and columns as documents (N=21,578) 
2. minhash signatures:
The HxN signature matrix: with H as the number of hash functions, N=21,578
3. candidate pairs:
For each document i, there should be a list of those documents j>i with which i needs to be compared
4. comparison of KNN search an linear search


## Implementation Notes

Note the differences in rows and columns for the input data and the output matrices
* Input: rows as documents
* Output: columns as documents

Your program should be able to accept some parameters:
* k in k-shingles
* Number of hash functions H




## Implementation

In [3]:
// Pre-Configured Spark Context in sc

println("Spark Entity:       " + spark)
println("Spark version:      " + spark.version)
println("Spark master:       " + spark.sparkContext.master)
println("Running 'locally'?: " + spark.sparkContext.isLocal)

Spark Entity:       org.apache.spark.sql.SparkSession@f26a3fa
Spark version:      2.2.0
Spark master:       local[*]
Running 'locally'?: true


In [3]:
val data = spark.sparkContext.textFile("./data/reut2-021.sgm")
data.take(100).foreach(println)

<!DOCTYPE lewis SYSTEM "lewis.dtd">
<REUTERS TOPICS="NO" LEWISSPLIT="TEST" CGISPLIT="TRAINING-SET" OLDID="20436" NEWID="21001">
<DATE>19-OCT-1987 15:37:46.03</DATE>
<TOPICS></TOPICS>
<PLACES></PLACES>
<PEOPLE></PEOPLE>
<ORGS></ORGS>
<EXCHANGES></EXCHANGES>
<COMPANIES></COMPANIES>
<UNKNOWN> 
&#5;&#5;&#5;F 
&#22;&#22;&#1;f2882&#31;reute
f f BC-CITYFED-FINANCI   10-19 0013</UNKNOWN>
<TEXT TYPE="BRIEF">&#2;
******<TITLE>CITYFED FINANCIAL CORP SAYS IT CUT QTRLY DIVIDEND TO ONE CENT FROM 10 CTS/SHR
</TITLE>Blah blah blah.
&#3;

</TEXT>
</REUTERS>
<REUTERS TOPICS="YES" LEWISSPLIT="TEST" CGISPLIT="TRAINING-SET" OLDID="20435" NEWID="21002">
<DATE>19-OCT-1987 15:35:53.55</DATE>
<TOPICS><D>crude</D><D>ship</D></TOPICS>
<PLACES><D>bahrain</D><D>iran</D><D>usa</D></PLACES>
<PEOPLE></PEOPLE>
<ORGS></ORGS>
<EXCHANGES></EXCHANGES>
<COMPANIES></COMPANIES>
<UNKNOWN> 
&#5;&#5;&#5;Y 
&#22;&#22;&#1;f2873&#31;reute
r f AM-GULF-PLATFORM   10-19 0101</UNKNOWN>
<TEXT>&#2;
<TITLE>HUGE OIL PLATFORMS DOT GULF LIK

data = ./data/reut2-021.sgm MapPartitionsRDD[3] at textFile at <console>:29


./data/reut2-021.sgm MapPartitionsRDD[3] at textFile at <console>:29

In [18]:
import scala.collection.mutable.ListBuffer
val data2 = spark.sparkContext.wholeTextFiles("./data/reut2-021.sgm")

var regex = """(?Us)<REUTERS.*NEWID="(\d*)".*>.*<TEXT.*>(.*)<\/TEXT>.*<\/REUTERS>""".r
var newsParse = data2.flatMap{
    case (path, string)  =>
    var list =  ListBuffer[Any]()
    var itr =  regex.findAllIn(string)
    for(m <- regex.findAllIn(string).matchData){
        val matchList = m.toSeq
//         val matchedData = e
//         list += matchedData
        list += (m(1),m(2))
    }
//     for(m <- regex.findAllIn(string).matchData;
//       e <- m.subgroups)
//       if(e!=null) list+=e
//     list.toSeq
}
// newsParse.take(5).foreach(println)
println(newsParse)

Name: Unknown Error
Message: <console>:48: error: value toSeq is not a member of scala.util.matching.Regex.Match
        val matchList = m.toSeq
                          ^
<console>:51: error: scala.util.matching.Regex.Match does not take parameters
        list += (m(1),m(2))
                  ^
<console>:51: error: scala.util.matching.Regex.Match does not take parameters
        list += (m(1),m(2))
                       ^
<console>:47: error: type mismatch;
 found   : Unit
 required: TraversableOnce[?]
    for(m <- regex.findAllIn(string).matchData){
          ^

StackTrace: 

### Task 1 - Find Min, Max and Count

In [5]:
val count = flattenData.map{case (k,v) => (k,1)}.reduceByKey((i, j) => i+j).collectAsMap()
count

Map(Global Active Power -> 2049280, Global Reactive Power -> 2049280, Global Intensity -> 2049280, Voltage -> 2049280)

In [6]:
val max = flattenData.reduceByKey{(i, j) => if (i>j) i else j}.collectAsMap()
max

Map(Global Active Power -> 11.122, Global Reactive Power -> 1.39, Global Intensity -> 48.4, Voltage -> 254.15)

In [7]:
val min = flattenData.reduceByKey{(i, j) => if (i<j) i else j}.collectAsMap()
min

Map(Global Active Power -> 0.076, Global Reactive Power -> 0.0, Global Intensity -> 0.2, Voltage -> 223.2)

### Task 2 - Mean & Standard Deviation

In [8]:
val average = flattenData.reduceByKey((i, j) => i+j).map{case (i, j) => (i,j/2049280)}.collectAsMap()
average

Map(Global Active Power -> 1.0916150365005446, Global Reactive Power -> 0.12371447630388221, Global Intensity -> 4.627759310588324, Voltage -> 240.8398579745135)

In [9]:
val std = flattenData.
    map{case (k,v) => (k, scala.math.pow(v-average(k),2))}.
    reduceByKey((i,j) => i+j).
    map{case (k,v) => (k, math.sqrt(v/count(k)))}.collectAsMap()
std

Map(Global Active Power -> 1.0572939031266613, Global Reactive Power -> 0.11272195204783488, Global Intensity -> 4.444395175407247, Voltage -> 3.239985888491343)

### Task 3 - Min-Max Normalization

In [10]:
val norm = flattenData.take(10).map{case (k,v) => (k, (v-min(k))/(max(k)-min(k)))}
norm

Array((Global Active Power,0.3747963063552418), (Global Reactive Power,0.30071942446043165), (Voltage,0.376090468497577), (Global Intensity,0.37759336099585067), (Global Active Power,0.4783632084012313), (Global Reactive Power,0.31366906474820144), (Voltage,0.33699515347334413), (Global Intensity,0.47302904564315357), (Global Active Power,0.4796306355241717), (Global Reactive Power,0.35827338129496406))

## Result 

In [11]:
object MyFunctions {
    def myprint(s: String): Unit = {
        println("For "+s+":")
        println("        Number of Meaningful Data - " + count(s))
        println("        Maximum Value - " + max(s))
        println("        Minimum Value - " + min(s))
        println("        Mean - " + average(s))
        println("        Standard Deviation - " + std(s) + "\n")
    }    
}

MyFunctions.myprint("Global Active Power")
MyFunctions.myprint("Global Reactive Power")
MyFunctions.myprint("Voltage")
MyFunctions.myprint("Global Intensity")


For Global Active Power:
        Number of Meaningful Data - 2049280
        Maximum Value - 11.122
        Minimum Value - 0.076
        Mean - 1.0916150365005446
        Standard Deviation - 1.0572939031266613

For Global Reactive Power:
        Number of Meaningful Data - 2049280
        Maximum Value - 1.39
        Minimum Value - 0.0
        Mean - 0.12371447630388221
        Standard Deviation - 0.11272195204783488

For Voltage:
        Number of Meaningful Data - 2049280
        Maximum Value - 254.15
        Minimum Value - 223.2
        Mean - 240.8398579745135
        Standard Deviation - 3.239985888491343

For Global Intensity:
        Number of Meaningful Data - 2049280
        Maximum Value - 48.4
        Minimum Value - 0.2
        Mean - 4.627759310588324
        Standard Deviation - 4.444395175407247



In [12]:
// For implicit conversions from RDDs to DataFrames
val spark2 = spark
import spark2.implicits._

object Norm {
    def AP(s: String): Double = {
        process(s, "Global Active Power")
    } 
    
    def RP(s: String): Double = {
        process(s, "Global Reactive Power")
    } 
    
    def V(s: String): Double = {
        process(s, "Voltage")
    } 
    
    def I(s: String): Double = {
        process(s, "Global Intensity")
    } 
    
    def process(s: String, k: String): Double = {
        var retVal = 0.0
        if(s == "?"){
            retVal = Double.NaN
        }else{
            retVal = (s.toDouble-min(k))/(max(k)-min(k))
        }
        retVal
    }
}

val dataDF = rows.
                map(_.split(";")).
                map(att => (att(0), att(1), Norm.AP(att(2)), Norm.RP(att(3)), Norm.V(att(4)), Norm.I(att(5)) )).
                toDF("Date", "Time", "Active_Power", "Reactive_Power","Voltage","Intensity")
                


In [17]:
dataDF.write.csv("./data/output")

In [14]:
dataDF.createOrReplaceTempView("records")
spark.sql("SELECT * FROM records WHERE date = '28/4/2007'").show(25)

+---------+--------+--------------------+-------------------+-------------------+--------------------+
|     Date|    Time|        Active_Power|     Reactive_Power|            Voltage|           Intensity|
+---------+--------+--------------------+-------------------+-------------------+--------------------+
|28/4/2007|00:00:00| 0.11696541734564549| 0.0618705035971223|0.31825525040387775| 0.11618257261410789|
|28/4/2007|00:01:00|  0.1171464783632084| 0.0618705035971223|0.32374798061389354| 0.11618257261410789|
|28/4/2007|00:02:00| 0.11732753938077133|0.06330935251798561| 0.3350565428109854| 0.11618257261410789|
|28/4/2007|00:03:00|  0.1171464783632084| 0.0618705035971223| 0.3295638126009697| 0.11618257261410789|
|28/4/2007|00:04:00| 0.11696541734564549| 0.0618705035971223| 0.3247172859450729| 0.11618257261410789|
|28/4/2007|00:05:00| 0.11696541734564549| 0.0618705035971223|0.32213247172859427| 0.11618257261410789|
|28/4/2007|00:06:00|  0.1171464783632084| 0.0618705035971223|0.3285945072

103820004 Michael Fu